In [1]:
# Importação de bibliotecas
import pandas as pd

In [2]:
# Variáveis utilizadas
dataset_name = 'risco_erosao_lavouras_temporarias'

raiz = '/home/eduardo/repositorios/IS_Agro/databases'

### Ouro

##### Lavouras Temporárias

In [4]:
# Tabela de área colhida (ha) de lavouras temporárias
df_temp = (
    pd.read_parquet(f'{raiz}/prata_sidra_tab1612_lavoura_temporaria_area_colhida.parquet')
    .query('data >= 2000')
    .reset_index()
    .assign(geocodigo_uf = lambda df: df.geocodigo// 100000)
    .groupby(['geocodigo_uf', 'data', 'cultura'])
    .agg({'area':'sum'})
    .rename_axis(index={'geocodigo_uf': 'geocodigo'})
    .convert_dtypes()
)

# Tabela de área colhida (ha) da primeira safra de lavouras temporárias
df_ps = (
    pd.read_parquet(f'{raiz}/ouro_volume_agricola_lavouras_primeira_safra.parquet')
    .assign(geocodigo_uf = lambda df: df.geocodigo// 100000)
    .groupby(['geocodigo_uf', 'data', 'cultura'])
    .agg({'valor':'sum'})
    .rename_axis(index={'geocodigo_uf': 'geocodigo'})
    .convert_dtypes()
)

# Tabela de área colhida (ha) da segunda safra de lavouras temporárias
df_ss = (
    pd.read_parquet(f'{raiz}/ouro_volume_agricola_lavouras_segunda_safra.parquet')
    .assign(geocodigo_uf = lambda df: df.geocodigo// 100000)
    .groupby(['geocodigo_uf', 'data', 'cultura'])
    .agg({'valor':'sum'})
    .rename_axis(index={'geocodigo_uf': 'geocodigo'})
    .convert_dtypes()
)

# Dados sobre área (por Mha) de Sistema Plantio Direto a nível estadual
df_spd = (
    pd.read_parquet(f'{raiz}/prata_SPD.parquet')
    .reset_index()
)

In [12]:
# Soja == 2713
df_soja = (
    df_temp
    .query('cultura == 2713 & area != 0')
    .reset_index()
    .drop(columns='cultura')
    .merge(
        df_ss
        .query('cultura == 2711 & valor != 0') # Dados da 2a. safra do milho
        .reset_index()
        .drop(columns='cultura'),
        how='left',
        on=['geocodigo', 'data']
    )
    .assign(
        area_SPD = lambda ha: ((ha.area / (ha.area + ha.valor)) * ha.area)
    )
    .assign(**{
        'Soja - Convencional - Monocultura': lambda ha: round(((ha.area - ha.area_SPD) / 1000000) * 0.5, 3), # 50% área soja - área soja SPD
        'Soja - Convencional - Sucessão': lambda ha: round(((ha.area - ha.area_SPD) / 1000000) * 0.5, 3), # 50% área soja - área soja SPD
        'Soja - Plantio Direto': lambda ha: round((ha.area_SPD / 1000000) * 0.7, 3), # 70% área soja SPD
        'Soja - SPD bem feito': lambda ha: round((ha.area_SPD / 1000000) * 0.3, 3) # 30% área soja SPD
    })
    .drop(columns=(['area', 'valor', 'area_SPD']))
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .convert_dtypes()
    #.groupby(['data', 'cultura']).agg({'valor': sum})
)

In [16]:
# Milho == 2711
df_milho = (
    df_ps
    .query('cultura == 2711 & valor != 0')
    .reset_index()
    .assign(**{
        'Milho - Monocultura (1a. safra)': lambda ha: round((ha.valor / 1000000) * 0.2, 3), # (27% x 25%, 20% de área)
        'Milho (1a. safra)': lambda ha: round((ha.valor / 1000000) * 0.8, 3), # (27% x 75%, 80% de área)
        # 'Milho - 2a. safra (verão)': lambda x: round((x.area/1000000)*0.16, 3), # (73% x 20%, 16% de área)
        # 'Milho - 2a. safra + braquiária': lambda x: round((x.area/1000000)*0.3, 3), # (73% x 40%, 30% de área)
        # 'Milho - 2a. safra - SPD': lambda x: round((x.area/1000000)*0.3, 3) # (73% x 40%, 30% de área)
    })
    .drop(columns=(['cultura', 'valor']))
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .convert_dtypes()
    #.groupby(['data', 'cultura']).agg({'valor': sum})
)

In [21]:
# Trigo == 2716 (revisão)
df_trigo = (
    df_ss
    .query('cultura == 2716')
    .reset_index()
    .assign(**{
        'Trigo - Convencional (2a. safra)': lambda ha: round((ha.valor / 1000000) * 0.1, 3), # 10% de área
        'Trigo - Plantio Direto (no-till - 2a. safra)': lambda ha: round((ha.valor/ 1000000) * 0.5, 3), # 50% de área
        'Trigo - Cultivo mínimo, SPD bem feito (2a. safra)': lambda ha: round((ha.valor / 1000000) * 0.4, 3) # 40% de área
    })
    .drop(columns=(['cultura', 'valor']))
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .convert_dtypes()
    #.groupby(['data', 'cultura']).agg({'valor': sum})
)

In [26]:
# Algodão Herbáceo == 2689 (revisão)
df_algodao_herb = (
    df_ps
    .query('cultura == 2698')
    .reset_index()
    .assign(**{
        'Algodão - Convencional - morro abaixo': lambda ha: round((ha.valor / 1000000) * 0.5, 4), # 40% de área
        'Algodão - Plano/sondulado, conservado': lambda ha: round((ha.valor / 1000000) * 0.5, 4) # 60% de área
    })
    .drop(columns=(['cultura', 'valor']))
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .convert_dtypes()
    #.groupby(['data', 'cultura']).agg({'valor': sum})
)

In [28]:
# Cana-de-açúcar == 2696
df_cana = (
    df_temp
    .query('cultura == 2696 & area != 0')
    .reset_index()
    .assign(**{
        'Cana - Cana-planta somente': lambda ha: round((ha.area / 1000000) * 0.2, 4), # 20% de área
        'Cana - Cana-planta + 4 cana-soca (5 colheitas)': lambda ha: round((ha.area / 1000000) * 0.8, 4) # 80% de área
    })
    .drop(columns=(['cultura', 'area']))
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .convert_dtypes()
    #.groupby(['data', 'cultura']).agg({'valor': sum})
)


In [51]:
# Mandioca == 2708
df_mandioca = (
    df_temp
    .query('cultura == 2708 & area != 0')
    .reset_index()
    .assign(**{
        'Mandioca - Convencional - solo coberto (entressafra)': lambda ha: round(ha.area / 1000000, 4)
    })
    .drop(columns=(['cultura', 'area']))
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .convert_dtypes()
    #.groupby(['data', 'cultura']).agg({'valor': sum})
)

In [35]:
# Arroz == 2692
df_arroz = (
    df_temp
    .query('cultura == 2692 & area != 0')
    .reset_index()
    .assign(**{
        'Arroz irrigado': lambda ha: round((ha.area/1000000) * 0.8, 3)
    })
    .drop(columns=(['cultura', 'area']))
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .convert_dtypes()
    #.groupby(['data', 'cultura']).agg({'valor': sum})
)

In [80]:
# Cereais
# Total de culturas temporárias 1a safra - (soja + milho 1a. safra + arroz irrigado + mandioca + cana + algodão 1a. safra)
df_cereais = (
    df_temp
    .query('area != 0')
    .reset_index()
    .groupby(['geocodigo', 'data'])
    .agg({'area':'sum'})
    .reset_index()
    .merge(
        df_temp
        .query('cultura == 2713 & area != 0'), # Soja
        how='left',
        on=['geocodigo', 'data'],
        suffixes=['_total', '_soja']
    )
    .merge(
        df_ps
        .query('cultura == 2711 & valor != 0'), # Milho 1a. safra
        how='left',
        on=['geocodigo', 'data'],
        #suffixes=['_total', '_milho']
    )
    .merge(
        df_arroz # Arroz irrigado
        .assign(valor = lambda x: x.valor * 1000000) # Retornando valor para ha para equivaler aos demais
        .drop(columns='cultura'),
        how='left',
        on=['geocodigo', 'data'],
        suffixes=['_milho', '_arroz']
    )
    .merge(
        df_temp
        .query('cultura == 2708 & area != 0'), # Mandioca
        how='left',
        on=['geocodigo', 'data'],
        #suffixes=['', '_mandioca']
    )
    .merge(
        df_temp
        .query('cultura == 2696 & area != 0'), # Cana-de-açúcar
        how='left',
        on=['geocodigo', 'data'],
        suffixes=['_mandioca', '_cana']
    )
    .merge(
        df_ps
        .query('cultura == 2698'), # Algodão 1a. safra
        how='left',
        on=['geocodigo', 'data'],
        suffixes=['_', '_algodao']
    )
    .fillna(0)
    .assign(**{
        'cereais_total': lambda n: n.area_total - (n.area_soja + n.valor_milho + n.valor_arroz + n.area_mandioca + n.area_cana + n.valor),
        'Cereais (arroz e feijão) - Convencional, sem cobertura': lambda x: x.cereais_total * 0.3,
        'Cereais - Convencional': lambda x: x.cereais_total * 0.4,
        'Cereais - Sistema Plantio Direto': lambda x: x.cereais_total * 0.3
    })
    .drop(columns={'area_total', 'area_soja', 'valor_milho', 'valor_arroz', 'area_mandioca', 'area_cana', 'valor', 'cereais_total'})
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .assign(valor = lambda ha: round(ha.valor/1000000, 2))
    .convert_dtypes()
    .reset_index(drop=True)
)

In [86]:
df_final = (
    pd.concat([
        df_soja,
        df_milho,
        df_trigo,
        df_algodao_herb,
        df_cana,
        df_mandioca,
        df_arroz,
        df_cereais
    ])
    .sort_values(['geocodigo', 'data'])
    .reset_index(drop=True)
)

In [88]:
df_ouro = (
    df_final
    .rename(columns={
        'geocodigo': 'state',
        'valor': 'value',
        'data': 'date',
        'cultura': 'label'
        })
    .assign(country = 76)
    .filter(['country', 'state', 'date', 'label', 'value'])
)

df_ouro.to_csv(f'{raiz}/ouro_{dataset_name}.csv', index=False)